# Initialization

In [1]:
from logging import ERROR
import os
from pathlib import Path
import signal
from subprocess import CalledProcessError
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces
from pyboolnet import log
log.setLevel(ERROR)

# mpbn init
import mpbn

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO NumExpr defaulting to 8 threads.


<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet
def bench_pyboolnet(models, timeout=180, number=1, max_output=1000):
    for model in models:
        print(model)        
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                primes = bnet2primes(model + ".bnet")
                tspaces = compute_trap_spaces(primes, "min", max_output=max_output)
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError, Exception):
                total_time += timeout
            except MemoryError:
                print ("MemoryError")
                total_time += timeout
            signal.alarm(0)
        
        os.system("killall -9 BNetToPrime_linux64")
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [3]:
# compute trapspaces using mpbn
def bench_mpbn(models, timeout=180, number=1, max_output=1000):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                mbn = mpbn.load(model + ".bnet")
                tspaces = list(mbn.attractors(limit=max_output))
                nb_tspaces = len(tspaces)
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except AssertionError:
                if i == 0:
                    print("Non-monotonic")
            except TimeOutException:
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

In [4]:
# compute trapspaces using trappist
def bench_trappist(models, timeout=180, number=1, max_output=1000, **kwargs):
    for model in models:
        print(model)
        successes = 0
        total_time = 0
        nb_tspaces = 0
        for i in range(number):
            signal.alarm(timeout)
            try:
                start = time.perf_counter()
                tspaces = trappist.compute_trap_spaces(model + ".bnet", max_output=max_output, time_limit=timeout, **kwargs)
                nb_tspaces = len(list(tspaces))
                end = time.perf_counter()
                total_time += end - start
                successes += 1
            except (TimeOutException, CalledProcessError):
                total_time += timeout
            signal.alarm(0)
        if successes > 0:
            if successes < number:
                print(f"{number - successes} failures…")
            print(f"{nb_tspaces} trap spaces {total_time/number:.2f}\n===")
        else:
            print("DNF\n===")

## Selected big models

Collected from various bibliographic sources (see article for details)

In [7]:
models = [str(Path().absolute() / "TCS-data-set/Selected" / name[:-5]) for name in os.listdir("TCS-data-set/Selected") if name.endswith(".bnet")]

In [17]:
bench_pyboolnet(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/macrophage_polarization
1000 trap spaces 0.54
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Cacace_TdevModel_2nov2020
28 trap spaces 5.67
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Pancreatic_Cancer
1000 trap spaces 0.11
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Mycobacterium_tuberculosis
1000 trap spaces 0.42
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Leishmania
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT_Mechanosensing_TGFbeta
492 trap spaces 11.28
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/DNA_damage_adaptation
16 trap spaces 0.33
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Executable_file_for_cholocystokinin_model_BCC
1000 trap spaces 0.36
===
/home/giang-trinh/git_repo/tr

In [18]:
bench_mpbn(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/macrophage_polarization
1000 trap spaces 1.09
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Cacace_TdevModel_2nov2020
Non-monotonic
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Pancreatic_Cancer
1000 trap spaces 0.36
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Mycobacterium_tuberculosis
1000 trap spaces 2.36
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Leishmania
1000 trap spaces 2.56
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT_Mechanosensing_TGFbeta
492 trap spaces 0.78
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/DNA_damage_adaptation
Non-monotonic
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Executable_file_for_cholocystokinin_model_BCC
1000 trap spaces 2.99
===
/home/giang-trinh/git_re

In [19]:
bench_trappist(models)

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/macrophage_polarization
1000 trap spaces 0.27
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Cacace_TdevModel_2nov2020
28 trap spaces 0.08
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Pancreatic_Cancer
1000 trap spaces 0.06
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Mycobacterium_tuberculosis
1000 trap spaces 0.44
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Leishmania
1000 trap spaces 0.46
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT_Mechanosensing_TGFbeta
492 trap spaces 0.35
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/DNA_damage_adaptation
16 trap spaces 0.05
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Executable_file_for_cholocystokinin_model_BCC
1000 trap spaces 0.37
===
/home/giang-trinh/gi

In [20]:
bench_trappist(models, method='sat')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/macrophage_polarization
1000 trap spaces 0.84
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Cacace_TdevModel_2nov2020
28 trap spaces 0.06
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Pancreatic_Cancer
1000 trap spaces 0.17
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Mycobacterium_tuberculosis
1000 trap spaces 4.91
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Leishmania
1000 trap spaces 5.62
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT_Mechanosensing_TGFbeta
492 trap spaces 0.69
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/DNA_damage_adaptation
16 trap spaces 0.02
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Executable_file_for_cholocystokinin_model_BCC
1000 trap spaces 4.81
===
/home/giang-trinh/gi

In [8]:
bench_trappist(models, method='cp')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/macrophage_polarization
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Cacace_TdevModel_2nov2020
28 trap spaces 7.51
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Pancreatic_Cancer
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Mycobacterium_tuberculosis
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Leishmania
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT_Mechanosensing_TGFbeta
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/DNA_damage_adaptation
16 trap spaces 3.91
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Executable_file_for_cholocystokinin_model_BCC
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT
268

In [9]:
bench_trappist(models, method='ilp')

/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/macrophage_polarization
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Cacace_TdevModel_2nov2020
28 trap spaces 23.15
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Pancreatic_Cancer
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Mycobacterium_tuberculosis
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Leishmania
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT_Mechanosensing_TGFbeta
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/DNA_damage_adaptation
16 trap spaces 5.33
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/Executable_file_for_cholocystokinin_model_BCC
Timeout
DNF
===
/home/giang-trinh/git_repo/trap-spaces-as-siphons/TCS-data-set/Selected/EMT
Ti